# Frizione dinamica con [Rebound](https://rebound.readthedocs.io/en/latest/)

### Indice
- Intro teorica frizione dinamica (da test-luca quasi scritto tutto)
- Test per funzionamento base di rebound
    - Basic: aggiungere particelle con certa posizione e velocità 
    - Come usare il widget per la visualizzazione
    - Esempi: [self-gravitating disc](https://rebound.readthedocs.io/en/latest/c_examples/selfgravity_disc/), [cometa CG](https://rebound.readthedocs.io/en/latest/ipython_examples/Churyumov-Gerasimenko/)
- Ricerca di un buon setup per studiare DF:
    - Box periodica con gas di particelle (destinate a collassare), poi aggiunto BH con determinata velocità: ma condizioni al contorno introducono effetti spuri
    - Tubo lungo dove il BH non fa in tempo a incontrare i muri
    - Idea vincente: galleria del vento
- Ottimizzazione del setup: stelle continuamente sostituite da stelle immesse da destra, con un tasso controllato
- Variamo mBH, v stelle (costante, MB), $\rho$ stelle, mantenendo log $\Lambda$ costante
- Su setup semplice (gas in box periodica, senza BH) variamo configurazione integratori, tree, softening, opening angle, teniamo parametri uguali ma numero di particelle variabile

# Test parametri di rebound


Vediamo di fare un setup semplice, gas in box periodiche (+ non periodiche?) e di vedere come cambia 

[Link prova](#sezione2)

<a id='sezione2'></a>
# LA sezione due